In [54]:
#Install necessary programs to the Jupyter notebook

#!pip install gmaps
#!jupyter nbextension enable --py gmaps
#!pip install beautifulsoup4
#!pip install google

In [55]:
#importing Modules & Api-Keys

import pandas as pd
import gmaps as gmaps
import random
import numpy as np
import sys
import requests
import json
from pprint import pprint
from googlesearch import search 

sys.path.append('../..')
from config import google_api_key
from config import aw_weather_api_key

gmaps.configure(api_key=google_api_key)

In [56]:
#importing the most updated CSV 

csv_file = pd.read_csv("winedata_coord_type_2019JAN21.csv")
csv_file.head(1)

,Unnamed: 0,Unnamed: 0.1,Title,Variety,Year,Price,Grade,Winery,Province,Region,Country,Taster,Taster_Twitter,Description,Latitude,Longitude,Type
0,0,1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,2011,15.0,87,Quinta dos Avidagos,Douro,NaN,Portugal,Roger Voss,@vossroger,"This is ripe and fruity, a wine that is smooth...",41.512019,-5.492255,Red


In [57]:
#Removing "Unnamed:0" column

df = csv_file.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
df.head(1)

,Title,Variety,Year,Price,Grade,Winery,Province,Region,Country,Taster,Taster_Twitter,Description,Latitude,Longitude,Type
0,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,2011,15.0,87,Quinta dos Avidagos,Douro,NaN,Portugal,Roger Voss,@vossroger,"This is ripe and fruity, a wine that is smooth...",41.512019,-5.492255,Red


In [58]:
#Dropping a decimal from "Price" column and re-naming the column

price = [int(x) for x in df["Price"]]
df["Price"] = price
df.head(2)

,Title,Variety,Year,Price,Grade,Winery,Province,Region,Country,Taster,Taster_Twitter,Description,Latitude,Longitude,Type
0,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,2011,15,87,Quinta dos Avidagos,Douro,NaN,Portugal,Roger Voss,@vossroger,"This is ripe and fruity, a wine that is smooth...",41.512019,-5.492255,Red
1,Quinta do Portal 2012 Verdelho and Sauvignon B...,Portuguese White,2012,15,87,Quinta do Portal,Douro,NaN,Portugal,Roger Voss,@vossroger,Verdelho and Sauvignon Blanc hangs together ea...,41.512019,-5.492255,White


In [59]:
#Renaming the Type of Wines

df["Type"] = df["Type"].replace(["Red"], "Red Wine")
df["Type"] = df["Type"].replace(["White"], "White Wine")
df["Type"] = df["Type"].replace(["Blush"], "Blush Wine")
df["Type"] = df["Type"].replace(["Sparkling"], "Sparkling Wine")

wine_types = df["Type"]
wine_types.value_counts()

Red Wine          53691
White Wine        27015
Other             14000
Blush Wine         5344
Sparkling Wine     1035
Name: Type, dtype: int64

In [60]:
#This contains functions that are useful for this program.

def find_title(dataframe, row):
    return (dataframe.loc[row,: "Title"])["Title"]

def find_year(dataframe, row):
    return (dataframe.loc[row,: "Year"])["Year"]

def find_country(dataframe, row):
    return (dataframe.loc[row,: "Country"])["Country"]

def find_description(dataframe, row):
    return (dataframe.loc[row,: "Description"])["Description"]

def find_price(dataframe, row):
    return (dataframe.loc[row,: "Price"])["Price"]

def print_wine_info(dataframe, row):
    print(f'     "{find_title(dataframe, row)}".')
    print(f'     The wine is from {find_country(dataframe, row)}, made in {find_year(dataframe, row)}, and may cost you ${find_price(dataframe, row)}.')
    print(f'     A reviewer says "{find_description(dataframe, row)}".')

def print_remaining_wine(dataframe):
    print(f'Our current database contains {len(dataframe["Title"])} wine-reviews from {len((dataframe["Country"]).unique())} countries.')
    print(f'These wine are from {min(dataframe["Year"])} to {max(dataframe["Year"])} with a price range of ${min(dataframe["Price"])} to ${max(dataframe["Price"])}.')
    
def pick_a_wine(dataframe):
    return random.randrange(len(dataframe["Title"]))

def median_score(dataframe):
    return np.median(dataframe["Grade"])

def summarize(dataframe):
    return pd.DataFrame({"Total Number of Wine Reviews" : [len(dataframe['Title'].unique())],
                "Total Number of Wineries": [len(dataframe['Winery'].unique())],
                "Total Number of Provinces": [len(dataframe['Province'].unique())],
                "Total Number of Countries": [len(dataframe['Country'].unique())],
                "Total Number of Years" : [len(dataframe['Year'].unique())],
                "Total Number of Tasters": [len(dataframe['Taster'].unique())]
                         #,"Total Number of Types": [len(dataframe['Type'].unique())]
                        })


In [61]:
#introducing our program

print(f'----------------------------')
print(f'Welcome to our Wine-Database.')
print(f'We can help you find a wine from {len(df["Title"])} wine-reviews from {len((df["Country"]).unique())} countries.')
print(f'These wine are from {min(df["Year"])} to {max(df["Year"])} with a price range of ${min(df["Price"])} to ${max(df["Price"])}.')
print(f'We can either tailor a wine to your liking, or randomly generate a highly-rated wine.')


----------------------------
Welcome to our Wine-Database.
We can help you find a wine from 101085 wine-reviews from 12 countries.
These wine are from 2004 to 2016 with a price range of $4 to $3300.
We can either tailor a wine to your liking, or randomly generate a highly-rated wine.


In [62]:
#Our program

#First Answer = Yes or No
#Second Answer = Type, or Skip
#Third Answer = Price or Skip
#Fourth Answer = Country or Skip
#Fifth Answer = Flavor


wine_types = df["Type"].unique()

first_answer = input("Type 'Yes' if you want us to help you find a wine, 'No' if you just want to see a random list of wine: ")
if first_answer.lower() == "yes":
    print("")
    print(df["Type"].unique())
    second_answer = input("What is your choice of wine type? If you have no preference, type 'Skip': ")
    if second_answer.lower() == "skip":
        print("")
        print_remaining_wine(df)
        third_answer = input("What is your price limit?” Either type the dollar amount, or type 'Skip' if you have too much money: ")
        
        if third_answer.isdigit() == True:
            new_df = df.loc[df["Price"] < int(third_answer) + 0.1]
            print("")
            print_remaining_wine(new_df)
            print("")
            print(new_df["Country"].unique())
            fourth_answer = input("What country's wine do you prefer? If you have no preference, type 'skip': " )
    
            if fourth_answer in new_df["Country"].unique():
                second_new_df = new_df.loc[new_df["Country"] == fourth_answer]
                third_new_df = second_new_df.loc[second_new_df["Grade"] >= median_score(second_new_df)]
                third_new_df = third_new_df.reset_index(drop=True)
                choice_wine = pick_a_wine(third_new_df)
                print("")
                print_wine_info(third_new_df, choice_wine)
                lat = (third_new_df.loc[choice_wine,: "Latitude"])["Latitude"]
                long = (third_new_df.loc[choice_wine,: "Longitude"])["Longitude"]
                location = [(lat, long)]
                wine_year = (third_new_df.loc[choice_wine,: "Year"])["Year"]
                query = find_title(third_new_df, choice_wine)
            
            elif fourth_answer.lower() == "skip":
                second_new_df = new_df.loc[new_df["Grade"] >= median_score(new_df)]
                second_new_df = second_new_df.reset_index(drop=True)
                choice_wine = pick_a_wine(second_new_df)
                print("")
                print_wine_info(second_new_df, choice_wine)
                lat = (second_new_df.loc[choice_wine,: "Latitude"])["Latitude"]
                long = (second_new_df.loc[choice_wine,: "Longitude"])["Longitude"]
                location = [(lat, long)]
                wine_year = (second_new_df.loc[choice_wine,: "Year"])["Year"]
                query = find_title(second_new_df, choice_wine)
        
        elif third_answer.lower() == "skip":
            print("")
            print_remaining_wine(df)
            print("")
            print(new_df["Country"].unique())
            fourth_answer = input("What country's wine do you prefer? If you have no preference, type 'skip': " )
    
            if fourth_answer in new_df["Country"].unique():
                new_df = df.loc[df["Country"] == fourth_answer]
                second_new_df = new_df.loc[new_df["Grade"] >= median_score(new_df)]
                second_new_df = second_new_df.reset_index(drop=True)
                choice_wine = pick_a_wine(second_new_df)
                print("")
                print_wine_info(second_new_df, choice_wine)
                lat = (second_new_df.loc[choice_wine,: "Latitude"])["Latitude"]
                long = (second_new_df.loc[choice_wine,: "Longitude"])["Longitude"]
                location = [(lat, long)]
                wine_year = (second_new_df.loc[choice_wine,: "Year"])["Year"]
                query = find_title(second_new_df, second_wine)
                
            elif fourth_answer.lower() == "skip":
                new_df = df.loc[df["Grade"] >= median_score(df)]
                new_df = new_df.reset_index(drop=True)
                choice_wine = pick_a_wine(new_df)
                print("")
                print_wine_info(new_df, choice_wine)
                lat = (new_df.loc[choice_wine,: "Latitude"])["Latitude"]
                long = (new_df.loc[choice_wine,: "Longitude"])["Longitude"]
                location = [(lat, long)]
                wine_year = (new_df.loc[choice_wine,: "Year"])["Year"]
                query = find_title(new_df, choice_wine)
    
    elif second_answer in df["Type"].unique():
        new_df = df.loc[df["Type"] == second_answer]
        print("")
        print_remaining_wine(new_df)
        third_answer = input("What is your price limit?” Either type the dollar amount, or type 'skip' if you have too much money: ")

        if third_answer.isdigit() == True:
            second_new_df = df.loc[df["Price"] < int(third_answer) + 0.1]
            print("")
            print_remaining_wine(second_new_df)
            print("")
            print(new_df["Country"].unique())
            fourth_answer = input("What country's wine do you prefer? If you have no preference, type 'skip': " )
            
            if fourth_answer in new_df["Country"].unique():
                third_new_df = second_new_df.loc[second_new_df["Country"] == fourth_answer]
                fourth_new_df = third_new_df.loc[third_new_df["Grade"] >= median_score(third_new_df)]
                fourth_new_df = fourth_new_df.reset_index(drop=True)
                choice_wine = pick_a_wine(fourth_new_df)
                print("")
                print_wine_info(fourth_new_df, choice_wine)
                lat = (fourth_new_df.loc[choice_wine,: "Latitude"])["Latitude"]
                long = (fourth_new_df.loc[choice_wine,: "Longitude"])["Longitude"]
                location = [(lat, long)]
                wine_year = (fourth_new_df.loc[choice_wine,: "Year"])["Year"]
                query = find_title(fourth_new_df, choice_wine)
                

            elif fourth_answer.lower() == "skip":
                third_new_df = second_new_df.loc[second_new_df["Grade"] >= median_score(second_new_df)]
                third_new_df = third_new_df.reset_index(drop=True)
                choice_wine = pick_a_wine(third_new_df)
                print("")
                print_wine_info(third_new_df, choice_wine)
                lat = (third_new_df.loc[choice_wine,: "Latitude"])["Latitude"]
                long = (third_new_df.loc[choice_wine,: "Longitude"])["Longitude"]
                location = [(lat, long)]
                wine_year = (third_new_df.loc[choice_wine,: "Year"])["Year"]
                query = find_title(third_new_df, choice_wine)
        elif third_answer.lower() == "skip":
            print("")
            print_remaining_wine(new_df)
            print("")
            print(new_df["Country"].unique())
            fourth_answer = input("What country's wine do you prefer? If you have no preference, type 'skip': " )
            
            if fourth_answer in new_df["Country"].unique():
                second_new_df = new_df.loc[new_df["Country"] == fourth_answer]
                third_new_df = second_new_df.loc[second_new_df["Grade"] >= median_score(second_new_df)]
                third_new_df = third_new_df.reset_index(drop=True)
                choice_wine = pick_a_wine(third_new_df)
                print("")
                print_wine_info(third_new_df, choice_wine)
                lat = (third_new_df.loc[choice_wine,: "Latitude"])["Latitude"]
                long = (third_new_df.loc[choice_wine,: "Longitude"])["Longitude"]
                location = [(lat, long)]
                wine_year = (third_new_df.loc[choice_wine,: "Year"])["Year"]
                query = find_title(third_new_df, choice_wine)
                
            elif fourth_answer.lower() == "skip":
                #second_new_df = new_df.loc[new_df["Country"] == fourth_answer]
                second_new_df = new_df.loc[new_df["Grade"] >= median_score(new_df)]
                seconnd_new_df = second_new_df.reset_index(drop=True)
                choice_wine = pick_a_wine(second_new_df)
                print("")
                print_wine_info(second_new_df, choice_wine)
                lat = (second_new_df.loc[choice_wine,: "Latitude"])["Latitude"]
                long = (second_new_df.loc[choice_wine,: "Longitude"])["Longitude"]
                location = [(lat, long)]
                wine_year = (second_new_df.loc[choice_wine,: "Year"])["Year"]
                query = find_title(second_new_df, choice_wine)

elif first_answer.lower() == "no":
    
    rdnumber = random.randrange(len(df["Title"])+1)
    new_df = df.loc[df["Grade"] >= median_score(df)]
    new_df = new_df.reset_index(drop=True)
    choice_wine = pick_a_wine(new_df)
    print("")
    print_wine_info(new_df, choice_wine)
    lat = (new_df.loc[choice_wine,: "Latitude"])["Latitude"]
    long = (new_df.loc[choice_wine,: "Longitude"])["Longitude"]
    location = [(lat, long)]
    wine_year = (new_df.loc[choice_wine,: "Year"])["Year"]
    query = find_title(df, choice_wine)
    
             
                

Type 'Yes' if you want us to help you find a wine, 'No' if you just want to see a random list of wine: Yes

['Red Wine' 'White Wine' 'Other' 'Blush Wine' 'Sparkling Wine']
What is your choice of wine type? If you have no preference, type 'Skip': Red Wine

Our current database contains 53691 wine-reviews from 12 countries.
These wine are from 2004 to 2016 with a price range of $4 to $3300.
What is your price limit?” Either type the dollar amount, or type 'skip' if you have too much money: 100

Our current database contains 98162 wine-reviews from 12 countries.
These wine are from 2004 to 2016 with a price range of $4 to $100.

['Portugal' 'US' 'Spain' 'Italy' 'France' 'Germany' 'Argentina' 'Chile'
 'Australia' 'Austria' 'South Africa' 'New Zealand']
What country's wine do you prefer? If you have no preference, type 'skip': Spain

     "Granbazán 2015 Etiqueta Verde Albariño (Rías Baixas)".
     The wine is from Spain, made in 2015, and may cost you $19.
     A reviewer says "Aromas of f

In [64]:
#Winery Geo-location

fig = gmaps.figure()
print(location) 
marker_layer = gmaps.marker_layer(location)
fig.add_layer(marker_layer)
fig


[(42.575055400000004, -8.133855800000001)]


Figure(layout=FigureLayout(height='420px'))

In [65]:
#Getting Weather Information

weather_url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?q="

if lat < 0: #Southern Hemisphere
    weather_info = requests.get(f"{weather_url}{lat},{long}&key={aw_weather_api_key}&date={wine_year}-02-01&enddate={wine_year}-02-31&tp=24&format=json").json()
    month = "February"
elif lat > 0: #Northern hemisphere
    weather_info = requests.get(f"{weather_url}{lat},{long}&key={aw_weather_api_key}&date={wine_year}-08-01&enddate={wine_year}-08-31&tp=24&format=json").json()
    month = "August"
    
fahrenheit = []
precip = []

x = 0

for weather in weather_info["data"]["weather"]:
    fahrenheit.append(weather_info["data"]["weather"][x]["hourly"][0]["tempF"])
    precip.append(weather_info["data"]["weather"][x]["hourly"][0]["precipMM"])
    x = x + 1

fahrenheit = [int(x) for x in fahrenheit]
precip = [int(float(x)) for x in precip]
avg_temp = np.average(fahrenheit)
avg_temp = round(avg_temp,2)
avg_precip = np.average(precip)


weather_information = pd.DataFrame({f'Avg Daily Temp for Summer (°F)' : [avg_temp],
                                    f'Avg Daily Precipiation for Summer (mm)': [avg_precip]})
                                    
weather_information


,Avg Daily Temp for Summer (°F),Avg Daily Precipiation for Summer (mm)
0,85.45,0.903226


In [66]:
#Google Searches

print(query)
for j in search(query, tld="com", num=5, stop=1, pause=2): 
    print(j)

Granbazán 2015 Etiqueta Verde Albariño (Rías Baixas)
https://www.winemag.com/buying-guide/granbazan-2015-etiqueta-verde-albarino-rias-baixas
https://www.wine.com/product/granbazan-etiqueta-verde-albarino-2015/159365
https://www.skurnik.com/sku/albarino-rias-baixas-etiqueta-verde-granbazan-3-2-2/
https://www.cellartracker.com/wine.asp?iWine=2367668
http://www.riasbaixaswines.com/winery/granbazan/wine/2708/
https://www.wine-searcher.com/find/agro+de+bazan+grand+etiqueta+verde+rias+baixas+galicia+spain
https://nosgustaelvino.com/2016/06/03/granbazan-albarino-etiqueta-verde-2015/
https://www.alko.fi/en/products/914932/Granbaz-n-Etiqueta-Verde-Albari-o-2017
https://www.cellartracker.com/wine.asp?iWine=2127530
https://www.wine-searcher.com/find/agro+de+bazan+grand+amber+rias+baixas+galicia+spain
https://www.youtube.com/watch?v=MueN6yor2lA
